# 01 - EDA: Computer Marketplace Dataset

## Objective

This notebook performs an initial exploration of `db_computers_2025_raw.csv` - a dataset of computer listings from a marketplace.

### Goals

1. **Understand the target variable** (price) - its distribution, range, and potential transformations
2. **Inspect distributions and missing values** across all columns
3. **Identify core features** for the initial model:
   - Product type (Tipo de producto)
   - Brand (Marca) and Series (Serie)
   - Processor (Procesador_Procesador)
   - Graphics card (Gráfica_Tarjeta gráfica)
   - RAM, storage, screen specs
4. **Prepare observations** for the feature engineering notebook

### Important Conventions

- **Original Spanish column names are preserved** - we do NOT rename or translate columns
- All comments and markdown are in English for readability
- Engineered features (created in later notebooks) will start with underscore: `_cpu_mark`, `_ram_gb`, etc.

---

## 1. Imports and Display Options

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
from pathlib import Path

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Seaborn style
sns.set_theme(style='whitegrid', palette='deep')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

print("Libraries loaded successfully!")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")

## 2. Load Data

In [ ]:
# Define paths (relative to notebooks/ folder)
DATA_DIR = Path('../data')

# Main dataset
computers_path = DATA_DIR / 'db_computers_2025_raw.csv'
columns_info_path = DATA_DIR / 'db_computers_columns_names.txt'

# Load the main dataset
# Note: Using utf-8-sig to handle BOM, index_col=0 then reset_index per course instructions
df = pd.read_csv(
    computers_path,
    encoding='utf-8-sig',
    index_col=0
).reset_index()

# Rename the old index column if needed
if 'index' in df.columns:
    df = df.rename(columns={'index': 'id_original'})

print(f"Dataset shape: {df.shape}")
print(f"  - Rows (computer listings): {df.shape[0]:,}")
print(f"  - Columns: {df.shape[1]}")

In [ ]:
# Preview the first few rows
# Note: Original Spanish column names are preserved exactly as they appear in the CSV
df.head()

In [ ]:
# List all columns to see what's available
print("All columns in the dataset:\n")
for i, col in enumerate(df.columns, 1):
    print(f"{i:3d}. {col}")

## 3. Basic Info and Data Types

In [ ]:
# Get basic info about the dataset
df.info()

In [ ]:
# Check unique values for key categorical columns
# These are columns we suspect will be important for prediction

key_columns = [
    'Tipo de producto',           # Product type (laptop, desktop, etc.)
    'Serie',                       # Product series/line
    'Tipo',                        # Type (Laptop, etc.)
    'Procesador_Fabricante del procesador',  # CPU manufacturer (Intel, AMD, Apple)
    'Gráfica_Fabricante de la tarjeta gráfica',  # GPU manufacturer
    'Sistema operativo_Sistema operativo',  # OS
]

print("Unique value counts for key categorical columns:\n")
for col in key_columns:
    if col in df.columns:
        print(f"{col}:")
        print(f"  Unique values: {df[col].nunique()}")
        print(f"  Missing: {df[col].isna().sum()} ({df[col].isna().mean()*100:.1f}%)")
        print()

In [ ]:
# Value counts for product type - this is crucial for understanding our dataset
if 'Tipo de producto' in df.columns:
    print("Product types (Tipo de producto):\n")
    print(df['Tipo de producto'].value_counts(dropna=False))

In [ ]:
# Value counts for 'Tipo' column
if 'Tipo' in df.columns:
    print("Type (Tipo):\n")
    print(df['Tipo'].value_counts(dropna=False))

In [ ]:
# CPU manufacturers
if 'Procesador_Fabricante del procesador' in df.columns:
    print("CPU Manufacturers (Procesador_Fabricante del procesador):\n")
    print(df['Procesador_Fabricante del procesador'].value_counts(dropna=False))

## 4. Target Variable Exploration: Price (Precio)

The price information is stored in the `Precio_Rango` column as a string range (e.g., "1.026,53 € – 2.287,17 €").

For modeling, we'll need to extract a numeric value from this range. For now, let's explore it as-is.

In [ ]:
# Identify the price column
precio_col = 'Precio_Rango'

print(f"Price column: {precio_col}")
print(f"\nSample values:")
print(df[precio_col].head(10))

In [ ]:
def extraer_precio_numerico(precio_str):
    """
    Extract numeric price from a range string.
    For ranges like "1.026,53 € – 2.287,17 €", returns the midpoint.
    For single prices, returns that price.
    
    Spanish format: periods for thousands, commas for decimals.
    """
    if pd.isna(precio_str) or not isinstance(precio_str, str):
        return np.nan
    
    import re
    
    # Find all price patterns (number with Spanish format)
    # Pattern: digits with optional periods, comma, digits
    pattern = r'([\d.]+,\d{2})'
    matches = re.findall(pattern, precio_str)
    
    if not matches:
        return np.nan
    
    # Convert Spanish format to float
    precios = []
    for m in matches:
        # Remove thousand separators (.), replace decimal comma with period
        num_str = m.replace('.', '').replace(',', '.')
        precios.append(float(num_str))
    
    # Return midpoint if range, otherwise single value
    if len(precios) == 2:
        return (precios[0] + precios[1]) / 2
    elif len(precios) == 1:
        return precios[0]
    else:
        return np.mean(precios)

# Extract numeric prices (for EDA only - proper extraction in features.py)
df['_precio_eda'] = df[precio_col].apply(extraer_precio_numerico)

print(f"Extracted numeric prices:")
print(f"  Valid prices: {df['_precio_eda'].notna().sum()} / {len(df)}")
print(f"  Missing: {df['_precio_eda'].isna().sum()}")

In [ ]:
# Price statistics
print("Price Statistics (in €):\n")
print(df['_precio_eda'].describe())

In [ ]:
# Price distribution visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
ax1 = axes[0]
df['_precio_eda'].hist(bins=50, ax=ax1, edgecolor='black', alpha=0.7)
ax1.set_xlabel('Price (€)')
ax1.set_ylabel('Frequency')
ax1.set_title('Price Distribution (Histogram)')
ax1.axvline(df['_precio_eda'].median(), color='red', linestyle='--', label=f"Median: {df['_precio_eda'].median():,.0f}€")
ax1.axvline(df['_precio_eda'].mean(), color='orange', linestyle='--', label=f"Mean: {df['_precio_eda'].mean():,.0f}€")
ax1.legend()

# Box plot
ax2 = axes[1]
df.boxplot(column='_precio_eda', ax=ax2)
ax2.set_ylabel('Price (€)')
ax2.set_title('Price Distribution (Box Plot)')

plt.tight_layout()
plt.show()

# Observations on price distribution
print("\nObservations:")
print("- The distribution appears right-skewed (long tail of high prices)")
print("- There are potential outliers at the high end (expensive gaming/workstation PCs)")
print("- A log-transform might help normalize the distribution for modeling")

In [ ]:
# Log-transformed price distribution (preview for future consideration)
fig, ax = plt.subplots(figsize=(10, 5))

df['_log_precio'] = np.log1p(df['_precio_eda'])
df['_log_precio'].hist(bins=50, ax=ax, edgecolor='black', alpha=0.7, color='green')
ax.set_xlabel('Log(Price + 1)')
ax.set_ylabel('Frequency')
ax.set_title('Log-Transformed Price Distribution')

plt.tight_layout()
plt.show()

print("Note: Log-transform makes the distribution more symmetric.")
print("Consider using log-price as target, then exp() to get final predictions.")

## 5. Missing Values Overview

Understanding missing values is crucial for:
- Deciding which features to include in the model
- Choosing imputation strategies
- Identifying potentially unreliable columns

In [ ]:
# Calculate missing values per column
missing_counts = df.isna().sum().sort_values(ascending=False)
missing_pct = (df.isna().sum() / len(df) * 100).sort_values(ascending=False)

# Create a summary DataFrame
missing_df = pd.DataFrame({
    'Missing Count': missing_counts,
    'Missing %': missing_pct,
    'Present %': 100 - missing_pct
})

# Show columns with most missing values
print("Columns with MOST missing values (top 30):\n")
print(missing_df.head(30))

In [ ]:
# Show columns with LEAST missing values (most complete)
print("Columns with LEAST missing values (most complete):\n")
print(missing_df.tail(30))

In [ ]:
# Visualize missing values for key feature columns
# Focus on columns we expect to use for modeling

key_feature_cols = [
    'Precio_Rango',
    'Título',
    'Tipo de producto',
    'Serie',
    'Procesador_Procesador',
    'Procesador_Fabricante del procesador',
    'Procesador_Número de núcleos del procesador',
    'RAM_Memoria RAM',
    'RAM_Tipo de RAM',
    'Disco duro_Capacidad de memoria SSD',
    'Disco duro_Tipo de disco duro',
    'Gráfica_Tarjeta gráfica',
    'Gráfica_Fabricante de la tarjeta gráfica',
    'Gráfica_Memoria gráfica',
    'Pantalla_Diagonal de la pantalla',
    'Pantalla_Resolución de pantalla',
    'Tipo',
]

# Filter to columns that exist
key_feature_cols = [c for c in key_feature_cols if c in df.columns]

key_missing = missing_df.loc[key_feature_cols].sort_values('Missing %', ascending=True)

fig, ax = plt.subplots(figsize=(10, 8))
key_missing['Present %'].plot(kind='barh', ax=ax, color='steelblue')
ax.set_xlabel('Data Completeness (%)')
ax.set_title('Data Completeness for Key Feature Columns')
ax.set_xlim(0, 100)

# Add percentage labels
for i, v in enumerate(key_missing['Present %']):
    ax.text(v + 1, i, f'{v:.1f}%', va='center', fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# Categorize columns by missing data levels
very_sparse = missing_df[missing_df['Missing %'] >= 80].index.tolist()  # >80% missing
sparse = missing_df[(missing_df['Missing %'] >= 50) & (missing_df['Missing %'] < 80)].index.tolist()
moderate = missing_df[(missing_df['Missing %'] >= 20) & (missing_df['Missing %'] < 50)].index.tolist()
dense = missing_df[missing_df['Missing %'] < 20].index.tolist()  # <20% missing

print(f"Very Sparse (>80% missing): {len(very_sparse)} columns")
print(f"Sparse (50-80% missing): {len(sparse)} columns")
print(f"Moderate (20-50% missing): {len(moderate)} columns")
print(f"Dense (<20% missing): {len(dense)} columns")

print("\n--- Dense columns (likely useful for initial model) ---")
for col in dense:
    print(f"  {col}: {missing_df.loc[col, 'Present %']:.1f}% present")

### Missing Values Observations

**Dense columns (low missingness, likely useful):**
- `Título` (Title) - Always present, can extract brand info
- `Precio_Rango` - Target variable, mostly present
- `Tipo de producto` - Product category (gaming, multimedia, etc.)
- `Tipo` - Laptop vs Desktop classification

**Moderate missingness (need imputation):**
- `Procesador_Procesador` - CPU name, will need fuzzy matching to benchmarks
- `RAM_Memoria RAM` - RAM info, extractable with regex
- Screen-related columns

**Sparse columns (might drop or use carefully):**
- Many detailed spec columns are very sparse
- Some columns only apply to specific product types (e.g., battery for laptops only)

## 6. First Thoughts on Core Features

Based on domain knowledge about computer pricing, here are the features we expect to be most predictive:

In [ ]:
# Core features we plan to use for the initial model
# Note: We'll create engineered versions of many of these with _ prefix

posibles_features_core = [
    # Category/Type
    'Tipo de producto',           # Product type (gaming, multimedia, professional)
    'Tipo',                        # Laptop vs Desktop
    'Serie',                       # Product series (often indicates tier)
    
    # Processor
    'Procesador_Procesador',       # CPU model - will map to _cpu_mark benchmark
    'Procesador_Fabricante del procesador',  # Intel, AMD, Apple
    'Procesador_Número de núcleos del procesador',  # Core count
    
    # Graphics
    'Gráfica_Tarjeta gráfica',     # GPU model - will map to _gpu_mark benchmark
    'Gráfica_Fabricante de la tarjeta gráfica',  # NVIDIA, AMD, Intel
    'Gráfica_Memoria gráfica',     # VRAM
    
    # Memory and Storage
    'RAM_Memoria RAM',             # RAM - will extract to _ram_gb
    'RAM_Tipo de RAM',             # DDR4, DDR5, LPDDR5X
    'Disco duro_Capacidad de memoria SSD',  # SSD - will extract to _ssd_gb
    'Disco duro_Tipo de disco duro',  # SSD, HDD, hybrid
    
    # Display
    'Pantalla_Diagonal de la pantalla',  # Screen size - will extract to _tamano_pantalla_pulgadas
    'Pantalla_Resolución de pantalla',   # Resolution
    'Pantalla_Tasa de actualización de imagen',  # Refresh rate (important for gaming)
]

print("Core features for initial model:")
for i, feat in enumerate(posibles_features_core, 1):
    if feat in df.columns:
        present_pct = (1 - df[feat].isna().mean()) * 100
        print(f"{i:2d}. {feat}: {present_pct:.1f}% present")
    else:
        print(f"{i:2d}. {feat}: NOT FOUND")

In [ ]:
# Explore RAM column
print("RAM_Memoria RAM - Value distribution:\n")
if 'RAM_Memoria RAM' in df.columns:
    print(df['RAM_Memoria RAM'].value_counts(dropna=False).head(20))
    print("\n-> RAM seems mostly present, values like '16 GB RAM', '8 GB RAM', etc.")
    print("-> Will need to extract numeric value to _ram_gb")

In [ ]:
# Explore processor column
print("Procesador_Procesador - Sample values:\n")
if 'Procesador_Procesador' in df.columns:
    print(df['Procesador_Procesador'].value_counts(dropna=False).head(20))
    print("\n-> Many unique CPU models (Intel Core i7-13700H, AMD Ryzen 7, Apple M3, etc.)")
    print("-> Will need to map to benchmark scores using db_cpu_raw.csv and fuzzy matching")

In [ ]:
# Explore GPU column
print("Gráfica_Tarjeta gráfica - Sample values:\n")
if 'Gráfica_Tarjeta gráfica' in df.columns:
    print(df['Gráfica_Tarjeta gráfica'].value_counts(dropna=False).head(20))
    print("\n-> Some GPU fields are sparse (integrated graphics often not listed)")
    print("-> Will need fuzzy matching to db_gpu_raw.csv for _gpu_mark")

In [ ]:
# Explore SSD capacity column
print("Disco duro_Capacidad de memoria SSD - Sample values:\n")
if 'Disco duro_Capacidad de memoria SSD' in df.columns:
    print(df['Disco duro_Capacidad de memoria SSD'].value_counts(dropna=False).head(20))
    print("\n-> Values like '512 GB', '1.000 GB', '256 GB'")
    print("-> Will extract to numeric _ssd_gb")

In [ ]:
# Explore screen size column
print("Pantalla_Diagonal de la pantalla - Sample values:\n")
if 'Pantalla_Diagonal de la pantalla' in df.columns:
    print(df['Pantalla_Diagonal de la pantalla'].value_counts(dropna=False).head(15))
    print("\n-> Values in cm like '39,624 cm', '35,56 cm'")
    print("-> Will convert to inches for _tamano_pantalla_pulgadas")

In [ ]:
# Price by product type - initial exploration
if 'Tipo de producto' in df.columns:
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Filter to types with enough samples
    tipo_counts = df['Tipo de producto'].value_counts()
    main_tipos = tipo_counts[tipo_counts >= 10].index
    
    df_plot = df[df['Tipo de producto'].isin(main_tipos)]
    
    df_plot.boxplot(column='_precio_eda', by='Tipo de producto', ax=ax)
    ax.set_xlabel('Tipo de producto')
    ax.set_ylabel('Price (€)')
    ax.set_title('Price Distribution by Product Type')
    plt.suptitle('')  # Remove automatic title
    plt.xticks(rotation=45, ha='right')
    
    plt.tight_layout()
    plt.show()
    
    print("\nObservation: Gaming laptops (Portátil gaming) tend to have higher prices.")
    print("Product type will be an important categorical feature.")

## 7. Summary and Next Steps

### What We Learned

1. **Dataset Size**: The dataset contains computer listings with ~140+ columns covering detailed specifications.

2. **Target Variable (Price)**:
   - Stored as a price range string in `Precio_Rango`
   - Right-skewed distribution with outliers at high end
   - Log-transform may help for modeling
   - Need to extract numeric value for `_precio_num`

3. **Missing Values**:
   - Many columns are very sparse (>80% missing)
   - Core columns like `Título`, `Tipo de producto`, `Precio_Rango` are mostly complete
   - Will need imputation strategy for moderately-sparse features

4. **Core Features Identified**:
   - **Product category**: `Tipo de producto`, `Tipo`, `Serie`
   - **Processor**: `Procesador_Procesador` → needs benchmark matching
   - **Graphics**: `Gráfica_Tarjeta gráfica` → needs benchmark matching
   - **Memory**: `RAM_Memoria RAM` → extract numeric GB
   - **Storage**: `Disco duro_Capacidad de memoria SSD` → extract numeric GB
   - **Screen**: `Pantalla_Diagonal de la pantalla` → convert to inches

5. **Key Observations**:
   - Gaming laptops (`Portátil gaming`) have higher prices
   - CPU/GPU models are varied and will need fuzzy matching to benchmark data
   - Many numeric values stored as strings with Spanish format (comma decimal)

### Open Questions

1. How to handle computers with integrated graphics (no dedicated GPU)?
2. Should we model different product types separately or together?
3. Which sparse columns are still worth including with careful imputation?
4. How to extract brand from title when `Marca` column is missing?

---

### Next Notebook: `02_feature_engineering.ipynb`

In the next notebook, we will:

1. **Create engineered features** using `src/features.py`:
   - `_precio_num`: Numeric price (midpoint of range)
   - `_cpu_mark`: CPU benchmark score from `db_cpu_raw.csv`
   - `_gpu_mark`: GPU benchmark score from `db_gpu_raw.csv`
   - `_ram_gb`: RAM in GB (numeric)
   - `_ssd_gb`: SSD capacity in GB (numeric)
   - `_tamano_pantalla_pulgadas`: Screen size in inches

2. **Implement missing value strategy**:
   - Decide which columns to keep vs drop
   - Choose imputation methods (median, mode, constant)

3. **Prepare clean feature table** for modeling (notebook 03)

In [ ]:
# Cleanup: Remove temporary columns created for EDA
eda_cols = ['_precio_eda', '_log_precio']
for col in eda_cols:
    if col in df.columns:
        df = df.drop(columns=[col])

print("EDA notebook complete!")
print(f"Next: 02_feature_engineering.ipynb")